# Welcome to Digital Earth Australia

> This tutorial introduces the DEA Python library for working with 30+ years of high resolution remote sensing data for Australia.

#### [Digital Earth Australia](https://www.ga.gov.au/dea) (DEA) is a digital platform that offer large amounts of Earth observation data that is ready to be analysed, covering continental Australia. DEA functionality is offered using a Python module, building on top and expanding the functionality that we have seen in Numpy, Matplotlib and XArray. To access the functionality of DEA we need to import the library. Then we declare a `Datacube` object that is going to be our point of access to the DEA data collections.

In [ ]:
%matplotlib inline

import datacube

dc = datacube.Datacube(app='welcome-to-dea')

print(dc)
print("---------------")
type(dc)

#### Digital Earth Australia offers a catalogue of data from a range of satellite sensors that offer the most recent images all the way back to 1986. The DEA data comes from the following satellites:

* [Landsat 5 TM](https://www.usgs.gov/land-resources/nli/landsat/landsat-5?qt-science_support_page_related_con=0#qt-science_support_page_related_con) (LS5 TM), operational between March 1984 and January 2013
* [Landsat 7 ETM+](https://www.usgs.gov/land-resources/nli/landsat/landsat-7?qt-science_support_page_related_con=0#qt-science_support_page_related_con) (LS7 ETM+), operational since April 1999
* [Landsat 8 OLI](https://www.usgs.gov/land-resources/nli/landsat/landsat-8?qt-science_support_page_related_con=0#qt-science_support_page_related_con) (LS8 OLI), operational since February 2013
* [Sentinel 2A MSI](https://sentinel.esa.int/web/sentinel/missions/sentinel-2) (S2A MSI), operational since June 2015
* [Sentinel 2B MSI](https://sentinel.esa.int/web/sentinel/missions/sentinel-2) (S2B MSI, operational since March 2017

#### Landsat missions are jointly operated by the United States Geological Survey (USGS) and National Aeronautics and Space Administration (NASA). Sentinel missions are operated by the European Space Agency (ESA). One major difference between the two programs is the spatial resolution: each Landsat pixel represents 30 x 30 m on the ground while each Sentinel-2 pixel represents 10 x 10 m to 60 x 60 m depending on the spectral band.

![Image](https://prd-wret.s3-us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/styles/full_width/public/thumbnails/image/dmidS2LS7Comparison.png)


#### The `Datacube` object that we have created has functionality attached to it. For example, we can get a list of the products available by doing:

In [ ]:
dc.list_products()#.head()

> Note: The `.head()` method in the previous cell limits the rown of the list to the first 10. You can try removing or commenting out this method using the `#` symbol to see the total number of rows although Jupyter will limit the size of the list. 

#### The [Sandbox Explorer](https://explorer.sandbox.dea.ga.gov.au/) is an online tool that documents the datasets available in the Sandbox environment. 

#### EPSG stands for European Petroleum Survey Group and this accronym commonly refers to the geodetic parameter database with standard codes for coordinate systems, datums, spheroids and units. This code is normally used to uniquely define the map projection used to generate a map. As you'll probably know there are different map projections with different characteristics, such as Mercator, Conic or Conformal.

#### Exercise: Can you find the EPSG code that DEA uses to store its data collections. Youn migth need to scroll towards the right side of the table to locate this column. 

* **a)** EPSG:4326
* **b)** EPSG:3577
* **c)** EPSG:2360

> Tip: If you are interested in knowing more about EPSG codes [here](https://spatialreference.org/) shows a full list of reference systems and descriptions.

In [ ]:
from check_answer import check_answer

# Substitute the ? symbols by either 'a', 'b' or 'c'
answ = 'b'

check_answer("8.1", answ)

#### Digital Earth Australia produces Analysis Ready Data (ARD) for each of the sensors listed above. The [ARD standard](http://ceos.org/ard/) for satellite data requires that data have undergone a number of processing steps, along with the creation of additional attributes for the data. DEA's ARD datasets include the following characteristics:

* **Geometric correction:** This includes establishing ground position, accounting for terrain (orthorectification) and ground control points, and assessing absolute position accuracy. 
Geometric calibration means that imagery is positioned accurately on the Earth's surface and stacked consistently so that sequential observations can be used to track meaningful change over time.
Adjustments for ground variability typically use a Digital Elevation Model (DEM).
* **Surface reflectance correction:** This includes adjustments for sensor/instrument gains, biases and offsets, include adjustments for terrain illumination and sensor viewing angle with respect to the pixel position on the surface.
Once satellite data is processed to surface reflectance, pixel values from the same sensor can be compared consistently both spatially and over time.
* **Observation attributes:** Per-pixel metadata such as quality flags and content attribution that enable users to make informed decisions about the suitability of the products for their use. For example, clouds, cloud shadows, missing data, saturation and water are common pixel level attributes.
* **Metadata:** Dataset metadata including the satellite, instrument, acquisition date and time, spatial boundaries, pixel locations, mode, processing details, spectral or frequency response and grid projection.


#### Optical sensors mounted on satellites measure the energy that comes from the sun and has been reflected by the Earth's surface. The sensor measures the energy in each of its spectral bands. There are many factors that affect the amount of energy that reaches the satellite sensor such as the solar and satellite angles relative to the surface and the atmospheric conditions. In order to analyse and compare images taken at different times, the data needs to be normalised into some standard conditions. Surface reflectance is one of this normalisations and is calculated using advanced physical models to correct the reflectance values taking into account atmospheric conditions, sun and satellite angles and local topography.

#### DEA offers to options for adjusted or normalised surface reflectance:

* **NBAR**: NBAR stands for *Nadir-corrected BRDF Adjusted Reflectance*, where BRDF stands for *Bidirectional reflectance distribution function*. The approach involves atmospheric correction to compute bottom-of-atmosphere radiance, and bi-directional reflectance modelling to remove the effects of topography and angular variation in reflectance. NBAR can be useful for analyses in extremely flat areas not affected by terrain shadow, and for producing attractive data visualisations that are not affected by NBART's nodata gaps (see below).

* **NBART**: NBART has the same features of NBAR but includes an additional *terrain illumination* reflectance correction that takes into account the surface topography. Terrain affects optical satellite images in a number of ways; for example, slopes facing the sun receive more sunlight and appear brighter compared to those facing away from the sun. This correction is performed with a Digital Surface Model (DSM). NBART is typically the default choice for performing analysis that are consistent through time. However, it can be introduce distortion in flat areas due to the noise in DSM data.

![Comparison between NBAR and NBART](data/nbar_nbart_animation.gif)

#### The animation above demonstrates how the NBART correction results in a significantly more two-dimensional looking image that is less affected by terrain illumination and shadow. Black pixels in the NBART image represent areas of deep terrain shadow that can't be corrected as they're determined not to be viewable by either the sun or the satellite. These are represented by -999 `nodata` values in the data.

> Note: Remember that remote sensing data is often stored as `uint16` to save space and `NaN` is only available for floating point types, that is why we need a value `-999` in this case to designate not-a-number data.

#### Exercise: In which cases can NBAR be preferred to NBART data?

* **a)** When dealing with images with clouds and shadows affecting the surface reflectance values.
* **b)** In areas with mountains and high relief to perform time-series analysis.
* **c)** In flat regions when we want to minimise the occurrence of no-data values.

In [ ]:
# Substitute the ? symbols by either 'a', 'b' or 'c'
answ = ?

check_answer("8.2", answ)

#### Now that know about the satellites and levels of processing in DEA, lets see how we can request some data.

#### Requesting data is done in a natural way by providing information about:
* What is the data collection that we want to request. **What**
* What area or spatial extent we want our image to cover. **Where**
* What is the time span we want to cover. **When**

#### We now need to pass all this information to the `dc.load()` function. We normally use a dictionary to describe our request as in the following example:

>Tip: See [this](https://datacube-core.readthedocs.io/en/latest/api/index.html) website for a full documentation of this and other DEA API functions

In [ ]:
query={'product': 'ga_ls8c_nbart_gm_cyear_3',                     # What
       'lat': (-35.25, -35.35), 'lon': (149.05, 149.17),          # Where
       'time':('2018-01-01', '2019-01-01')}                       # When

ds = dc.load(**query)

ds

>Tip: DEA uses XArray as its underlying data model and defines a simple querying interface to select satellite data which returns as `Dataset` objects.

#### We can use the plotting functionality in XArray to generate a true colour image of this data:

In [ ]:
ds[['red', 'green', 'blue']].isel(time=0).to_array().plot.imshow(robust=True, figsize=(6,6))

#### Think of a region anywhere in Australia and create a similar plot.

>Tip: Use a range of ~0.1 degrees in longitude and latitude to speed up the request and avoid running out of memory.

>Tip: To easily find out the coordinates of a place you can use http://geojson.io